# Transform data
In this notebook we perform transformations required to get required features for modelling.

In [0]:
# Set up dependencies
import numpy as np
import pandas as pd
from pyspark.ml.feature import Bucketizer
from pyspark.sql.functions import count, when, isnan, col
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName("PopHealthRisk").getOrCreate()

In [0]:
df = spark.read.parquet('/Volumes/pophealthrisk/pophealthrisk/pophealthrisk/LLCP2024.parquet', header=True, inferSchema=True)

# Transform data
First we will clean variables that contain values that are interpreted as missing or 0 values.

In [0]:
def clean_variable(df, col, null_vals):
    df = df.withColumn(
        col+'_clean',
            when(df[col].isin(null_vals), np.nan).otherwise(df[col]))
    return(df)

In [0]:
# check function works as expected
df=clean_variable(df,'_AGEG5YR',[14])
df.groupBy('_AGEG5YR_clean').count().show()


+--------------+-----+
|_AGEG5YR_clean|count|
+--------------+-----+
|           1.0|29692|
|           6.0|28968|
|           4.0|28804|
|           5.0|30899|
|           8.0|34936|
|           NaN| 8310|
|          10.0|47701|
|           7.0|31698|
|          13.0|41756|
|          11.0|44774|
|           2.0|23705|
|           9.0|43387|
|          12.0|36803|
|           3.0|26237|
+--------------+-----+



In [0]:
variables_to_clean={"EDUCA":[9],'INCOME3':[77,99], '_SMOKER3':[9], 'DRNKANY6':[7,9],'CHILDREN':[99],'HHADULT':[0,77,88,99],'_RFHLTH':[9]}
#BMI does not require cleaning
for v in variables_to_clean:
    df=clean_variable(df,v,variables_to_clean[v])

df = df.withColumn(
        'CHILDREN_clean_mod',
            when(df['CHILDREN_clean'].isin([88]), 0).otherwise(df['CHILDREN_clean']))

In [0]:
# check that modifying CHILDREN_clean worked
df.groupBy('CHILDREN_clean_mod').count().show()

+------------------+------+
|CHILDREN_clean_mod| count|
+------------------+------+
|              14.0|    13|
|              NULL|  5606|
|              23.0|     1|
|               0.0|336299|
|              32.0|     2|
|              22.0|     3|
|              29.0|     1|
|              18.0|     1|
|               1.0| 48206|
|               6.0|   695|
|              25.0|     1|
|              15.0|     5|
|               4.0|  5672|
|              41.0|     1|
|               5.0|  1854|
|               8.0|   127|
|              81.0|    12|
|              17.0|     2|
|              20.0|     2|
|              82.0|     3|
+------------------+------+
only showing top 20 rows


In [0]:
# use _RFHLTH as target variable, adjust to 0,1 for classifier
df=df.withColumn(
    'RFHLTH_adj',
    when(df['_RFHLTH_clean'] == 1, 0).otherwise(
        when(df['_RFHLTH_clean']==2,1).otherwise(df['_RFHLTH_clean'])))

In [0]:
# make number of conditions column
condition_cols=['CVDINFR4','CVDCRHD4','CVDSTRK3','ASTHMA3','CHCSCNC1','CHCOCNC1','CHCCOPD3','ADDEPEV3','CHCKDNY2','HAVARTH4','DIABETE4']
df = df.withColumn(
    'num_conditions',
    sum(
        when(col(c) == 1, 1).otherwise(0) # count conditions that are listed as 1
        for c in condition_cols
    )
)

Next we will calculate income relative to poverty threshold. To do this we first make new variables with children capped at 8 and adults at 9. 

In [0]:
df = df.withColumn(
    'CHILDREN_cap8',
        when(df['CHILDREN_clean_mod'] >8, 8).otherwise(df['CHILDREN_clean_mod'])
    )


In [0]:
df = df.withColumn(
    'ADULT_cap9',
    when(df['HHADULT_clean']>9, 9).otherwise(df['HHADULT_clean'])
)

In [0]:
# check that cap function works
df.groupBy('ADULT_cap9').count().show()

+--------------------+------+
|          ADULT_cap9| count|
+--------------------+------+
|                NULL| 81961|
|                 1.0| 96588|
|                 6.0|  2788|
|                 4.0| 26067|
|                 5.0|  8557|
|                 8.0|   402|
|5.397605346934028...|     1|
|                 7.0|   911|
|                 2.0|182543|
|                 9.0|  4096|
|                 3.0| 53756|
+--------------------+------+



Now we have capped variables, we read in the poverty data generated in the convert_census notebook and bin to match the BRFSS data. 

In [0]:
poverty_df = pd.read_csv('/Volumes/pophealthrisk/pophealthrisk/pophealthrisk/thresh24.csv')
poverty_df=poverty_df.rename(columns={'household':'ADULT_cap9', 'children':'CHILDREN_cap8','threshold':'poverty_threshold'})
bins=[0,10000,15000,20000,25000,35000,50000,75000,100000,150000,200000,np.inf]
poverty_df['poverty_threshold_conv']=pd.cut(poverty_df['poverty_threshold'],bins, labels=[x for x in range(1,12)])
poverty_df['poverty_threshold_conv']=pd.factorize(poverty_df['poverty_threshold_conv'])[0] + 2
poverty_df = spark.createDataFrame(poverty_df) # convert to pyspark df for join

In [0]:
# Now we left join thresholds to the data
df = df.join(poverty_df, on=['ADULT_cap9','CHILDREN_cap8'], how='left')

In [0]:
df.groupBy('poverty_threshold_conv').count().show()

+----------------------+------+
|poverty_threshold_conv| count|
+----------------------+------+
|                  NULL|144700|
|                     2| 79243|
|                     7| 16199|
|                     3|176715|
|                     5|  3586|
|                     4| 32908|
|                     6|  4319|
+----------------------+------+



In [0]:
# We can now calculate the income adjusted to the poverty threshold
df = df.withColumn(
    'income_adj_pov',
    df['INCOME3_clean']-df['poverty_threshold_conv']
)

# Select features and filter

In [0]:
df_min=df.select(["_AGEG5YR_clean","EDUCA_clean",'_BMI5', '_SMOKER3_clean', 'DRNKANY6_clean','INCOME3_clean','num_conditions','income_adj_pov','RFHLTH_adj'])

In [0]:
# Check types are as expected
df_min.dtypes

[('_AGEG5YR_clean', 'double'),
 ('EDUCA_clean', 'double'),
 ('_BMI5', 'double'),
 ('_SMOKER3_clean', 'double'),
 ('DRNKANY6_clean', 'double'),
 ('INCOME3_clean', 'double'),
 ('num_conditions', 'int'),
 ('income_adj_pov', 'double'),
 ('RFHLTH_adj', 'double')]

In [0]:
# Count NA entries
na_cols=["_AGEG5YR_clean","EDUCA_clean",'_BMI5', '_SMOKER3_clean', 'DRNKANY6_clean','INCOME3_clean','num_conditions','income_adj_pov','RFHLTH_adj']
df_min.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in na_cols]).show()

+--------------+-----------+-----+--------------+--------------+-------------+--------------+--------------+----------+
|_AGEG5YR_clean|EDUCA_clean|_BMI5|_SMOKER3_clean|DRNKANY6_clean|INCOME3_clean|num_conditions|income_adj_pov|RFHLTH_adj|
+--------------+-----------+-----+--------------+--------------+-------------+--------------+--------------+----------+
|          8310|       2363|43037|         32022|         43777|        87423|             0|        199183|      1310|
+--------------+-----------+-----+--------------+--------------+-------------+--------------+--------------+----------+



In [0]:
print(df_min.na.drop().count())
print(df_min.count())

229399
457670


We lose a significant proportion of rows (1/2) when we require all values to be present. The 2 major contributors are INCOME3 and income_adj_pov, which derives from missing INCOME3 values and missing children/adults. A more thorough treatment could attempt to impute some of these values. 

In [0]:
df_min=df_min.na.drop()

In [0]:
# This is a heavily skewed dataset, mostly healthy people
df_min.groupBy('RFHLTH_adj').count().show()

+----------+------+
|RFHLTH_adj| count|
+----------+------+
|       0.0|185479|
|       1.0| 43920|
+----------+------+

